# Advanced Numpy Techniques

General, user-friendly [documentation](https://docs.scipy.org/doc/numpy/index.html) with lots of examples.

Technical, "hard" [reference](https://docs.scipy.org/doc/numpy/reference/index.html#reference).

## What are they?

background, etc.

## Motivation

applications, interfacing, etc. used by many libraries (give ex)

why numpy? E.g., over lists? (give ex!)

when to use numpy

bug free, expressive

most people learn through assimilation/necessity. (slow learning curve). investing some concentrated time now will give you a payoff when you don't need it

In [ ]:
# TODO cool use case, motivate!

## Goals and Non-goals

Describe Numpy, with under-the-hood details as they are useful to api users

Gotchas

Point out a couple useful numpy functions

Key take away are not the individual functions (there's too many to keep in your head) but give you an overview of the API structure and divisions so you know where to look


Won't be focusing on multicore processing, GPU interfacing, or other particular libraries -- but numpy can do all these things!

# The NumPy Array

In [1]:
# High-level: what is a tensor; recursive structure
# What can numpy arrays contain?
# Array representation (gotcha: fortran)
# identity, aliasing

#https://docs.scipy.org/doc/numpy/reference/arrays.html

SyntaxError: invalid syntax (<ipython-input-1-13c0d6657841>, line 1)

# Indexing

In [ ]:
# interaction with array identity


# Array Manipulation

### Array Creation and Initialization

In [2]:
# https://docs.scipy.org/doc/numpy-dev/reference/routines.html#routines
# random intialization

SyntaxError: invalid syntax (<ipython-input-2-2244c11f74a2>, line 3)

### Array Composition

In [ ]:
# https://docs.scipy.org/doc/numpy/reference/routines.array-manipulation.html
# padding

### Array Transformations

In [ ]:
# give general perf pointers -- vectorize, vectorize, vectorize for clarity and efficiency!

# unary elementwise functions /ufunc (1-1)
# https://docs.scipy.org/doc/numpy/reference/ufuncs.html

# multi-ary elementwise

# reductions (many-1)

# (multi-ary) tensor-to-tensor (many-many) ffts, linear algebra, outer, etc. Can still vectorize, even on many-to-many

# discussion on what gets put in C (experiments showing ufunc stuff not parallelized)
# http://stackoverflow.com/questions/35500925/is-numpy-einsum-efficient-compared-to-fortran-or-c

In [ ]:
# einsum ultilmate generalization of matrix products
# https://obilaniu6266h16.wordpress.com/2016/02/04/einstein-summation-in-numpy/
# einsum
#http://ajcr.net/Basic-guide-to-einsum
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html

# Takeaways

Keeping stuff in C can be easy and expressive and maintainable. Vectorize as much as you can: express your logic at the heighest level you can. Keep the time spent in C. Can't optimize across numpy ops (like a C compiler would/numpy would). But do you need that? Can't parallelize except BLAS, but is it computaitonal or memory bandwidth limited?



In [ ]:
Maybe: sparse matrices, scipy acceleration